<a href="https://colab.research.google.com/github/Mohit1053/quant-lab/blob/master/notebooks/colab_A_pretrain_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook A: Pre-training + Transformer Training
**Run on Colab Pro+ H100** | Part 1 of 3 parallel sessions
- Pre-trains masked time-series encoder (BERT-like)
- Trains Transformer forecaster with multi-task heads

In [34]:
!git clone https://github.com/Mohit1053/quant-lab.git


Cloning into 'quant-lab'...
remote: Enumerating objects: 365, done.
remote: Counting objects: 100% (365/365), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 365 (delta 134), reused 357 (delta 130), pack-reused 0 (from 0)
Receiving objects: 100% (365/365), 197.42 KiB | 28.20 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [2]:
# ==============================
# 🚀 QUANT LAB – COLAB SETUP
# ==============================

import os
import sys
import subprocess
from pathlib import Path

REPO_URL = "https://github.com/Mohit1053/quant-lab.git"
CLONE_DIR = "/content/quant-lab"
DRIVE_DIR = Path("/content/drive/MyDrive/quant_lab")

# -------------------------------------------------
# 1️⃣ Reset Working Directory (prevents getcwd bug)
# -------------------------------------------------
os.chdir("/content")

# -------------------------------------------------
# 2️⃣ Clean Broken / Partial Repo (safe re-run)
# -------------------------------------------------
if os.path.exists(CLONE_DIR):
    print("🧹 Removing existing quant-lab directory...")
    subprocess.run(["rm", "-rf", CLONE_DIR])

# -------------------------------------------------
# 3️⃣ Clone Repository
# -------------------------------------------------
print("📥 Cloning repository...")
result = subprocess.run(
    ["git", "clone", REPO_URL, CLONE_DIR],
    capture_output=True,
    text=True
)

if result.returncode != 0:
    print("❌ Git Clone Failed:\n")
    print(result.stderr)
    raise RuntimeError("Repository clone failed.")

print("✅ Clone successful.")

# -------------------------------------------------
# 4️⃣ Change into Repo Directory
# -------------------------------------------------
os.chdir(CLONE_DIR)

# -------------------------------------------------
# 5️⃣ Install Package (editable mode)
# -------------------------------------------------
print("📦 Installing package...")
subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "-e", "."],
    check=True
)
print("✅ Installation complete.")

# -------------------------------------------------
# 6️⃣ Mount Google Drive
# -------------------------------------------------
print("💾 Mounting Google Drive...")
from google.colab import drive
drive.mount("/content/drive", force_remount=False)

# -------------------------------------------------
# 7️⃣ Create Persistent Directory Structure
# -------------------------------------------------
print("📁 Creating Drive directories...")
for d in [
    "data/raw",
    "data/cleaned",
    "data/features",
    "outputs/models/pretrained",
    "outputs/models/transformer",
    "outputs/mlruns",
]:
    (DRIVE_DIR / d).mkdir(parents=True, exist_ok=True)

print("✅ Drive directories ready.")

# -------------------------------------------------
# 8️⃣ GPU Information
# -------------------------------------------------
print("🖥️ Checking GPU...")
import torch

if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    bf16 = torch.cuda.is_bf16_supported()
    print(f"🚀 GPU: {gpu}")
    print(f"💾 Memory: {mem:.1f} GB")
    print(f"🧠 BF16 Support: {bf16}")
else:
    print("⚠️ WARNING: No GPU detected.")


🧹 Removing existing quant-lab directory...
📥 Cloning repository...
✅ Clone successful.
📦 Installing package...
✅ Installation complete.
💾 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Creating Drive directories...
✅ Drive directories ready.
🖥️ Checking GPU...
🚀 GPU: NVIDIA H100 80GB HBM3
💾 Memory: 85.0 GB
🧠 BF16 Support: True


In [36]:

!pip uninstall -y numpy pandas scipy scikit-learn
!pip install --no-cache-dir numpy==1.26.4 pandas==2.2.2 scipy==1.11.4 scikit-learn==1.4.2


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: scipy 1.16.3
Uninstalling scipy-1.16.3:
  Successfully uninstalled scipy-1.16.3
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 461.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 460.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 220.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 91.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into acco

In [3]:
# === DATA PIPELINE ===
# Check if Drive has cached data, otherwise download fresh
import shutil

drive_features = DRIVE_DIR / 'data/features/nifty50_features.parquet'
local_features = Path('data/features/nifty50_features.parquet')

if drive_features.exists():
    print("Loading cached features from Drive...")
    Path('data/features').mkdir(parents=True, exist_ok=True)
    shutil.copy(drive_features, local_features)
    # Also copy cleaned data
    if (DRIVE_DIR / 'data/cleaned/nifty50_cleaned.parquet').exists():
        Path('data/cleaned').mkdir(parents=True, exist_ok=True)
        shutil.copy(DRIVE_DIR / 'data/cleaned/nifty50_cleaned.parquet', 'data/cleaned/nifty50_cleaned.parquet')
else:
    print("Downloading fresh data from yfinance...")
    subprocess.run([sys.executable, 'scripts/ingest_data.py'], check=True)
    subprocess.run([sys.executable, 'scripts/compute_features.py'], check=True)

    # Cache to Drive
    for src in ['data/raw', 'data/cleaned', 'data/features']:
        src_p = Path(src)
        if src_p.exists():
            dst_p = DRIVE_DIR / src
            dst_p.mkdir(parents=True, exist_ok=True)
            for f in src_p.glob('*.parquet'):
                shutil.copy(f, dst_p / f.name)
    print("Data cached to Drive for other notebooks!")

import pandas as pd
df = pd.read_parquet(local_features)
print(f"Features: {df.shape[0]} rows, {df['ticker'].nunique()} tickers, {len([c for c in df.columns if c not in {'date','ticker','open','high','low','close','volume','adj_close'}])} features")

Loading cached features from Drive...
Features: 177187 rows, 49 tickers, 15 features


## Pre-training: Masked Time-Series Encoder
BERT-like pre-training with patch tokenization and 15% masking
- H100 config: d_model=256, 6 layers, 100 epochs, batch_size=128

In [4]:
# === PRE-TRAINING (H100 optimized) ===
import time
from quant_lab.utils.seed import set_global_seed
from quant_lab.utils.device import get_device
from quant_lab.data.datasets import TemporalSplit
from quant_lab.data.datamodule import QuantDataModule, DataModuleConfig
from quant_lab.data.storage.parquet_store import ParquetStore
from quant_lab.features.engine import FeatureEngine
from quant_lab.representation.masked_encoder import MaskedTimeSeriesEncoder, MaskedEncoderConfig
from quant_lab.representation.pretraining import PreTrainer, PretrainConfig
from quant_lab.representation.embedding_space import EmbeddingExtractor

set_global_seed(42)
device = get_device()

# Load features
store = ParquetStore(base_dir='data/features')
feature_df = store.load('nifty50_features')

engine = FeatureEngine(
    enabled_features=['log_returns', 'realized_volatility', 'momentum', 'max_drawdown'],
    windows={'short': [1, 5], 'medium': [21], 'long': [63]},
)
feature_cols = engine.get_feature_columns(feature_df)

# Use train+val for pre-training (more data)
split = TemporalSplit(train_end='2023-06-30', val_end='2023-06-30')
dm = QuantDataModule(
    feature_df, feature_cols, split,
    DataModuleConfig(sequence_length=63, batch_size=128, num_workers=2),
)
dm.setup()
train_loader = dm.train_dataloader()

# H100-optimized: larger model
encoder_config = MaskedEncoderConfig(
    num_features=dm.num_features,
    patch_size=5,
    d_model=256,
    nhead=8,
    num_encoder_layers=6,
    dim_feedforward=1024,
    dropout=0.1,
    mask_ratio=0.15,
)
model = MaskedTimeSeriesEncoder(encoder_config)
print(f"Encoder parameters: {model.count_parameters():,}")

# Pre-train
pt_config = PretrainConfig(
    epochs=100,
    learning_rate=1e-4,
    mask_ratio=0.15,
    mixed_precision=True,
    checkpoint_dir='outputs/models/pretrained',
)
pretrainer = PreTrainer(model, pt_config, device)

start = time.time()
history = pretrainer.fit(train_loader)
elapsed = time.time() - start
print(f"\nPre-training done in {elapsed/60:.1f} min | Final loss: {history['train_loss'][-1]:.6f}")

# Save model
model.save(Path('outputs/models/pretrained/masked_encoder.pt'))

# Extract embeddings
extractor = EmbeddingExtractor(model, device)
embeddings = extractor.extract(train_loader)
print(f"Embeddings shape: {embeddings.shape}")

# Save to Drive
import shutil
for f in Path('outputs/models/pretrained').glob('*'):
    shutil.copy(f, DRIVE_DIR / 'outputs/models/pretrained' / f.name)
print("Pre-trained model saved to Drive!")

2026-02-19 09:50:28 [info     ] using_gpu                      memory_gb=79.2 name='NVIDIA H100 80GB HBM3'
2026-02-19 09:50:28 [info     ] parquet_loaded                 cols=23 path=data/features/nifty50_features.parquet rows=177187
2026-02-19 09:50:28 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=3331 valid_samples=3080
2026-02-19 09:50:28 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=3331 valid_samples=3080
2026-02-19 09:50:28 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=3331 valid_samples=3080
2026-02-19 09:50:28 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=3331 valid_samples=3080
2026-02-19 09:50:28 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=3331 valid_samples=3080
2026-02-19 09:50:28 [debug    ] dataset_created                num_features=15 sequence_length=63 tot

## Transformer Forecaster Training
Multi-task heads: return distribution (Gaussian) + direction (3-class) + volatility
- H100 config: d_model=256, 6 layers, 100 epochs, batch_size=128

In [5]:
# === TRANSFORMER TRAINING (H100 optimized) ===
import time
from quant_lab.models.transformer.model import TransformerForecaster, TransformerConfig, MultiTaskLoss
from quant_lab.training.trainer import Trainer, TrainerConfig

set_global_seed(42)
device = get_device()

# Proper train/val split
split = TemporalSplit(train_end='2021-12-31', val_end='2023-06-30')
dm = QuantDataModule(
    feature_df, feature_cols, split,
    DataModuleConfig(sequence_length=63, target_col='log_return_1d', batch_size=128, num_workers=2),
)
dm.setup()
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()

# H100-optimized model
model_cfg = TransformerConfig(
    num_features=dm.num_features,
    d_model=256,
    nhead=8,
    num_encoder_layers=6,
    dim_feedforward=1024,
    dropout=0.1,
    activation='gelu',
    distribution_type='gaussian',
    direction_num_classes=3,
    direction_threshold=0.005,
    volatility_enabled=True,
    distribution_weight=1.0,
    direction_weight=0.3,
    volatility_weight=0.3,
)
model = TransformerForecaster(model_cfg)
loss_fn = MultiTaskLoss(model_cfg)
print(f"Transformer parameters: {model.count_parameters():,}")

# Trainer
trainer_config = TrainerConfig(
    epochs=100,
    learning_rate=1e-4,
    weight_decay=1e-5,
    warmup_steps=1000,
    max_grad_norm=1.0,
    patience=15,
    mixed_precision=True,
    checkpoint_dir='outputs/models/transformer',
)
trainer = Trainer(model=model, loss_fn=loss_fn, config=trainer_config, device=device)

start = time.time()
history = trainer.fit(train_loader, val_loader)
elapsed = time.time() - start

print(f"\nTraining done in {elapsed/60:.1f} min")
print(f"Final train loss: {history['train_loss'][-1]:.6f}")
if history['val_loss']:
    print(f"Final val loss: {history['val_loss'][-1]:.6f}")
    print(f"Best val loss: {min(history['val_loss']):.6f}")

# Save to Drive
model.save(Path('outputs/models/transformer/final_model.pt'))
for f in Path('outputs/models/transformer').glob('*'):
    shutil.copy(f, DRIVE_DIR / 'outputs/models/transformer' / f.name)
print("Transformer model saved to Drive!")

2026-02-19 10:11:25 [info     ] using_gpu                      memory_gb=79.2 name='NVIDIA H100 80GB HBM3'
2026-02-19 10:11:25 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=2961 valid_samples=2710
2026-02-19 10:11:25 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=2961 valid_samples=2710
2026-02-19 10:11:25 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=2961 valid_samples=2710
2026-02-19 10:11:25 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=2961 valid_samples=2710
2026-02-19 10:11:25 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=2961 valid_samples=2710
2026-02-19 10:11:25 [debug    ] dataset_created                num_features=15 sequence_length=63 total_rows=2961 valid_samples=2710
2026-02-19 10:11:25 [debug    ] dataset_created                num_features=15 sequence_length=

In [6]:
# === NOTEBOOK A SUMMARY ===
print("=" * 60)
print("NOTEBOOK A COMPLETE")
print("=" * 60)
print(f"Pre-training loss: {history['train_loss'][-1]:.6f}")
print(f"Models saved to Google Drive: {DRIVE_DIR / 'outputs/models'}")
print("\nFiles on Drive:")
for f in sorted((DRIVE_DIR / 'outputs/models').rglob('*')):
    if f.is_file():
        size_mb = f.stat().st_size / 1e6
        print(f"  {f.relative_to(DRIVE_DIR)}: {size_mb:.1f} MB")
print("=" * 60)

NOTEBOOK A COMPLETE
Pre-training loss: -3.136875
Models saved to Google Drive: /content/drive/MyDrive/quant_lab/outputs/models

Files on Drive:
  outputs/models/pretrained/masked_encoder.pt: 23.8 MB
  outputs/models/transformer/best.pt: 59.1 MB
  outputs/models/transformer/final_model.pt: 21.1 MB
  outputs/models/transformer/last.pt: 59.1 MB


In [7]:
import os
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

DRIVE_DIR = Path("/content/drive/MyDrive/quant_lab")

print(f"Files in Google Drive directory: {DRIVE_DIR / 'outputs/models'}")
if (DRIVE_DIR / 'outputs/models').exists():
    for f in sorted((DRIVE_DIR / 'outputs/models').rglob('*')):
        if f.is_file():
            size_mb = f.stat().st_size / 1e6
            print(f"  {f.relative_to(DRIVE_DIR)}: {size_mb:.1f} MB")
else:
    print("No output models directory found in Google Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files in Google Drive directory: /content/drive/MyDrive/quant_lab/outputs/models
  outputs/models/pretrained/masked_encoder.pt: 23.8 MB
  outputs/models/transformer/best.pt: 59.1 MB
  outputs/models/transformer/final_model.pt: 21.1 MB
  outputs/models/transformer/last.pt: 59.1 MB
